## OpenCV Text Detection (EAST text detector)

As we’ll learn, OpenCV’s text detector implementation of EAST is quite robust, capable of localizing text even when it’s blurred, reflective, or partially obscured:

In [ ]:
import os
import cv2

In [ ]:
# import the necessary packages
from imutils.video import VideoStream
from imutils.video import FPS
from imutils.object_detection import non_max_suppression
import numpy as np
import argparse
import imutils
import time
import cv2

In [ ]:
def decode_predictions(scores, geometry):
    # grab the number of rows and columns from the scores volume, then
    # initialize our set of bounding box rectangles and corresponding
    # confidence scores
    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []
 
    # loop over the number of rows
    for y in range(0, numRows):
        # extract the scores (probabilities), followed by the
        # geometrical data used to derive potential bounding box
        # coordinates that surround text
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]
 
        # loop over the number of columns
        for x in range(0, numCols):
            # if our score does not have sufficient probability,
            # ignore it
            if scoresData[x] < min_confidence:
                continue
 
            # compute the offset factor as our resulting feature
            # maps will be 4x smaller than the input image
            (offsetX, offsetY) = (x * 4.0, y * 4.0)
 
            # extract the rotation angle for the prediction and
            # then compute the sin and cosine
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)
 
            # use the geometry volume to derive the width and height
            # of the bounding box
            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]
 
            # compute both the starting and ending (x, y)-coordinates
            # for the text prediction bounding box
            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)
 
            # add the bounding box coordinates and probability score
            # to our respective lists
            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])
 
    # return a tuple of the bounding boxes and associated confidences
    return (rects, confidences)

In [ ]:
video = '12.mp4'
east = 'frozen_east_text_detection.pb'
min_confidence = 0.8
width = 320
height = 320

In [ ]:
# initialize the original frame dimensions, new frame dimensions,
# and ratio between the dimensions
(W, H) = (None, None)
(newW, newH) = (width, height)
(rW, rH) = (None, None)
 
# define the two output layer names for the EAST detector model that
# we are interested -- the first is the output probabilities and the
# second can be used to derive the bounding box coordinates of text
layerNames = [
    "feature_fusion/Conv_7/Sigmoid",
    "feature_fusion/concat_3"]
 
# load the pre-trained EAST text detector
print("[INFO] loading EAST text detector...")
net = cv2.dnn.readNet(east)

In [ ]:
vs = cv2.VideoCapture(video)

# start the FPS throughput estimator
fps = FPS().start()

In [ ]:
# loop over frames from the video stream
frame_count = 0
while True:
    text_boxes = 0
    success,frame = vs.read()
    frame_count += 1
    # resize the frame, maintaining the aspect ratio
    if frame is None:
        break
    
    frame = imutils.resize(frame, width=1000)
    orig = frame.copy()
    print(orig.shape)
    orig_x = orig.shape[1]//5
    orig_y = orig.shape[0]//5
 
    # if our frame dimensions are None, we still need to compute the
    # ratio of old frame dimensions to new frame dimensions
    if W is None or H is None:
        (H, W) = frame.shape[:2]
        rW = W / float(newW)
        rH = H / float(newH)
 
    # resize the frame, this time ignoring aspect ratio
    frame = cv2.resize(frame, (newW, newH))
    
    
    # construct a blob from the frame and then perform a forward pass
    # of the model to obtain the two output layer sets
    
    blob = cv2.dnn.blobFromImage(frame, 1.0, (newW, newH),
        (123.68, 116.78, 103.94), swapRB=True, crop=False)
    net.setInput(blob)
    (scores, geometry) = net.forward(layerNames)
 
    # decode the predictions, then  apply non-maxima suppression to
    # suppress weak, overlapping bounding boxes
    (rects, confidences) = decode_predictions(scores, geometry)
    boxes = non_max_suppression(np.array(rects), probs=confidences)
    
    
    # loop over the bounding boxes
    for (startX, startY, endX, endY) in boxes:
        
        # scale the bounding box coordinates based on the respective
        # ratios
        startX = int(startX * rW)
        startY = int(startY * rH)
        endX = int(endX * rW)
        endY = int(endY * rH)
         
        if startX and endX in range(orig_x,orig_x*4) and startY and endY in range(orig_y,orig_y*4):
        
            # draw the bounding box on the frame
            cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 255, 0), 2)
            cv2.rectangle(orig, (orig_x, orig_y), (orig_x*4, orig_y*4), (255, 255, 0), 2)
            text_boxes += 1
        
        else:
            continue
        
    # update the FPS counter
    fps.update()
 
    # show the output frame
    cv2.imshow("Text Detection", orig)
    key = cv2.waitKey(1) & 0xFF
 
    print(frame_count,text_boxes)
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
    
 
# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))


vs.release()
 
# close all windows
cv2.destroyAllWindows()
    

$ python text_detection_video.py --east frozen_east_text_detection.pb  
[INFO] loading EAST text detector...  
[INFO] starting video stream...  
[INFO] elasped time: 59.76  
[INFO] approx. FPS: 8.85  